In [1]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)  # 전체 모델

print('\n\nJust one layer:')
print(tinymodel.linear2)  # 레이어 1개

print('\n\nModel params:')
for param in tinymodel.parameters():  # 파라미터
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():  # 레이어1개의 파라미터
    print(param)

c:\Users\taeyong\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[ 0.0888,  0.0821,  0.0260,  ...,  0.0466, -0.0195, -0.0060],
        [-0.0715, -0.0558,  0.0809,  ..., -0.0296,  0.0846, -0.0134],
        [-0.0847,  0.0963,  0.0370,  ..., -0.0964, -0.0873,  0.0807],
        ...,
        [ 0.0484,  0.0958,  0.0576,  ...,  0.0574,  0.0544, -0.0454],
        [ 0.0922, -0.0171,  0.0508,  ...,  0.0209,  0.0730, -0.0354],
        [-0.0891,  0.0725, -0.0201,  ..., -0.0892, -0.0790, -0.0689]],
       requires_grad=True)
Parameter containing:
tensor([-0.0395, -0.0649,  0.0941,  0.0727, -0.0256,  0.0160,  0.0894, -0.0452,
         0.0895,  0.0790, -0.0695,  0.0263, -0.0595,  0.0515,  0.0245,  0.0935,
        -0.0592,  0.0238, -0.0670,  0.04

In [2]:
lin = torch.nn.Linear(3, 2)  # 선형 레이어
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')  # m개의 입력과 n개의 출력이 존재하면, 가중치 행열은 m x n 행렬
for param in lin.parameters():
    print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.2646, 0.8382, 0.4898]])


Weight and Bias parameters:
Parameter containing:
tensor([[-0.1076,  0.4995,  0.3844],
        [-0.3900, -0.4231,  0.1710]], requires_grad=True)
Parameter containing:
tensor([-0.3462,  0.4930], requires_grad=True)


Output:
tensor([[0.2323, 0.1189]], grad_fn=<AddmmBackward0>)


In [3]:
import torch.functional as F


class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)  # 1개의 이미지, 6개의 출력 피처, 5x5 커널
        self.conv2 = torch.nn.Conv2d(6, 16, 3)  # 6개의 피처 , 16개의 출력 피처, 3x3 커널
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension  # 선형 레이어이기 때문에 1차원으로 넣어야함
        self.fc2 = torch.nn.Linear(120, 84)  # 120 -> 84
        self.fc3 = torch.nn.Linear(84, 10)  # 84 -> 10

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))  # 차원 변경 for 선형레이어
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.3203, 0.1013, 0.0721, 0.8973, 0.0353, 0.0339],
         [0.0793, 0.2894, 0.8718, 0.4366, 0.2397, 0.0397],
         [0.4559, 0.1840, 0.3472, 0.6394, 0.7621, 0.5223],
         [0.1764, 0.2733, 0.9393, 0.8112, 0.7139, 0.6021],
         [0.2402, 0.6107, 0.2227, 0.3860, 0.0218, 0.9079],
         [0.8984, 0.7145, 0.5481, 0.3867, 0.3925, 0.3371]]])
tensor([[[0.8718, 0.8973],
         [0.9393, 0.9079]]])


In [6]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[13.0077, 24.6785, 17.9283,  7.2547],
         [ 5.6251, 12.5292, 15.4948,  5.4185],
         [17.6945,  7.0986, 20.3522, 16.3436],
         [ 9.8474, 23.7154, 15.8247,  8.8876]]])
tensor(13.8563)
tensor([[[-0.4230,  1.3989,  0.3451, -1.3210],
         [-0.9471,  0.6316,  1.3097, -0.9943],
         [ 0.4654, -1.6581,  0.9980,  0.1947],
         [-0.7986,  1.5471,  0.2124, -0.9610]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-4.4703e-08, grad_fn=<MeanBackward0>)


In [7]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)  # 확률적으로 뉴런 비활성화 - 과적합 방지
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.0000, 0.2970, 0.1130, 0.0000],
         [1.0278, 1.5434, 0.3014, 0.6118],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.9992, 1.0933, 1.1001]]])
tensor([[[1.0694, 0.2970, 0.1130, 0.4142],
         [1.0278, 1.5434, 0.0000, 0.0000],
         [0.5629, 0.6245, 0.6481, 0.4399],
         [0.7664, 0.0000, 1.0933, 1.1001]]])
